# Домашнее задание №2.

#### 1. *Сгенерировать датасет при помощи sklearn.datasets.make_regression и обучить линейную модель при помощи градиентного спуска и стохастического градиентного спуска. Построить графики среднеквадратичной ошибки от числа итераций для обоих методов на одном рисунке, сделать выводы о разнице скорости сходимости каждого из методов.*

In [2]:
import numpy as np
from sklearn import datasets
from matplotlib import pyplot as plt

%matplotlib inline

In [15]:
X_, y_, coef = datasets.make_regression(n_samples=10000,
                                       n_features=3,
                                       n_informative=3,
                                       n_targets=1,
                                       noise=2,
                                       coef=True,
                                       random_state=15)

In [16]:
coef

array([56.78437196, 23.7256946 , 44.40999046])

Стандартизация выборки X_:

Метод градиентного спуска:

Метод стохастического градиентного спуска:

Метод градиентного спуска mini-batch:

Визуализация сравнения среднеквадратичных ошибок разных методов в зависимости от итераций:

#### 2.* *Добавить в алгоритм градиентного спуска L1 регуляризацию. Сравнить влияние коэффициентов регуляризации L1 и L2 на величину весов для примера из урока (лучше графически).*

L1-регуляризация:

L2-регуляризация:

Визуальное сравнение коэффициентов регуляризации:

In [ ]:
w_list = []
l_list = []
error_list =[]

for l in np.logspace(start=-4, stop=1, num=20):
    l_list.append(l)
    w_new = eval_model_reg2(?)(X_st, 